In [1]:
import time
import os
from copy import deepcopy
from itertools import combinations

import pickle
from tqdm.notebook import tqdm
from multiprocessing import Pool, cpu_count

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from neurodsp.utils.norm import normalize_sig
import mne
from mne_bids import BIDSPath, read_raw_bids

from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support

import torch
from torch import nn

from braindecode.models import Deep4Net, ShallowFBCSPNet

# Decoding Inner Speech 

## Data

Below, the preprocessed data is loaded from the BIDs deriviatives directory. See Nieto et al., 2022 for detailed preprocessing steps.

In [2]:
# Get paths
dpath = './ds003626-download/derivatives/'

subjects = os.listdir(dpath)
subjects.sort()
subjects = {k: {} for k in subjects if k.startswith('sub')}

for sub in subjects.keys():
    
    sessions = os.listdir(dpath + '/' + sub)
    sessions = [i for i in sessions if i.startswith('ses')]

    for sess in sessions:

        subjects[sub][sess] = {
            'events': f'{dpath}/{sub}/{sess}/{sub}_{sess}_events.dat',
            'file': f'{dpath}/{sub}/{sess}/{sub}_{sess}_eeg-epo.fif'
        }
        

In [3]:
# Load epoched data
fs = 256
order = 100

n_subjects = 10
n_sessions = 3
n_channels = 128

sessions = ['ses-01' ,'ses-02', 'ses-03']

sigs = {}

for sub in subjects.keys():
    
    sigs[sub] = {}
    
    for sess in sessions:
        
        # Read epochs
        epochs = mne.read_epochs(subjects[sub][sess]['file'], verbose=False)
        
        # Trim signals to closest multiple of 100 (for even array reshaping)
        sigs[sub][sess] = epochs._data[:, :, :1100].astype(np.float32)
        
    

In [4]:
# Organize signals by trial type
trial_types = ['prounounced', 'inner', 'visualized']
training_data = {k: {'X': [], 'labels': []} for k in trial_types}

for sub in subjects.keys():
    
    for sess in sessions:
        
        events = np.load(subjects[sub][sess]['events'], allow_pickle=True)
        
        for i, trial in enumerate(trial_types):
        
            inds = np.where(events[:, 2] == i)[0]
        
        
            training_data[trial]['labels'].append(events[inds][:, 1])
            training_data[trial]['X'].append(sigs[sub][sess][inds])

## Training

Each subject is trained separately using a deep convolution network. The goal is to decode the true labels of the four cues: up, down, right, left. Below can be skipped if ran previously.

In [ ]:
kfolds = 5
n_channels = 128
n_samples = 1100

accuracy = np.zeros((2, 10, 5))

# Train models separately for each subject
for isub, sub in enumerate(subjects.keys()):

    # Load data
    training_data = {'X': [], 'labels': []}

    for sess in sessions:

        events = np.load(subjects[sub][sess]['events'], allow_pickle=True)

        inds = np.where(events[:, 2] == 1)[0]

        training_data['labels'].append(events[inds][:, 1])
        training_data['X'].append(sigs[sub][sess][inds])

    # Flatten across sessions
    X_array = np.concatenate(training_data['X'])
    y_array = np.concatenate(training_data['labels'])

    # Normalize
    for i in range(n_channels):
        X_array[:, i] = normalize(X_array[:, i])

    # Ensure even samples per fold
    n_per_fold = int(np.floor(len(X_array)/kfolds))

    X_array = X_array[:int(n_per_fold * kfolds)]
    y_array = y_array[:int(n_per_fold * kfolds)]
    
    # Randomly shuffle
    np.random.seed(0)
    inds = np.arange(len(X_array))
    np.random.shuffle(inds)

    X_array = X_array[inds]
    y_array = y_array[inds]

    # K-folds
    #X_array = X_array[:, :, int(fs):int(3.5 * fs)]
    X_array = X_array.reshape(kfolds, -1, 128, n_samples)
    y_array = y_array.reshape(kfolds, -1)

    train_inds = np.array(list(combinations(np.arange(kfolds), 4)))
    test_inds = np.arange(5)[::-1]

    # To tensors
    X_tensor = torch.from_numpy(X_array.astype(np.float32))
    y_tensor = torch.from_numpy(y_array)

    # Train-test loop
    for k, (itrain, itest) in enumerate(zip(train_inds, test_inds)):

        # Reshape into train and test
        X = X_tensor[itrain]
        X = X.reshape(-1, n_channels, n_samples)

        y = y_tensor[itrain]
        y = y.reshape(-1)

        # Reinitialize model
        model = Deep4Net(
            #128, 4, 640, 'auto',
            128, 4, 1100, 'auto',
            n_filters_time=48,
            n_filters_spat=24,
            filter_time_length=64,
            pool_time_length=3,
            pool_time_stride=3,
            n_filters_2=24,
            filter_length_2=2,
            n_filters_3=32,
            filter_length_3=2,
            n_filters_4=48,
            filter_length_4=2,
        )

        # Train ConvNet
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, amsgrad=True)

        n_epochs = 100
        losses = []
        best_loss = np.inf

        for i in tqdm(range(n_epochs), total=n_epochs):

            # Compute loss and backprop
            loss = criterion(model(X), y)
            loss.backward()

            # Track loss
            current_loss = float(loss)
            losses.append(current_loss)

            # Save state of best model
            if current_loss < best_loss:
                best_loss = current_loss
                best_model = deepcopy(model.to('cpu'))

            # Step
            optimizer.step()  

        # Save models
        torch.save(best_model, f'models/convnet_{sub}_best_kfold{str(k)}')
        torch.save(model, f'models/convnet_{sub}_last_kfold{str(k)}')

        # Test
        X_test = X_tensor[itest]
        X_test = X_test.reshape(-1, n_channels, n_samples)

        y_test = y_tensor[itest]
        y_test = y_test.reshape(-1)

        model = torch.load(f'models/convnet_{sub}_best_kfold{str(k)}')
        acc_best = len(torch.where(torch.argmax(model(X_test), axis=1) == y_test)[0]) / len(X_test)
        accuracy[0, isub, k] = acc_best

        model = torch.load(f'models/convnet_{sub}_last_kfold{str(k)}')
        acc_last = len(torch.where(torch.argmax(model(X_test), axis=1) == y_test)[0]) / len(X_test)
        accuracy[1, isub, k] = acc_last

## Performance

In [87]:
n_channels = 128
n_samples = 1100

kfolds = 5

# Macro
accuracy = np.zeros((10, 5))
precision = np.zeros((10, 5))
recall = np.zeros((10, 5))
fscore = np.zeros((10, 5))

# Per class metrics
accuracy_class = np.zeros((10, 5, 4))
precision_class = np.zeros((10, 5, 4))
recall_class = np.zeros((10, 5, 4))
fscore_class = np.zeros((10, 5, 4))

# Train models separately for each subject
for isub, sub in enumerate(subjects.keys()):

    # Load data
    training_data = {'X': [], 'labels': []}

    for sess in sessions:

        events = np.load(subjects[sub][sess]['events'], allow_pickle=True)

        inds = np.where(events[:, 2] == 1)[0]

        training_data['labels'].append(events[inds][:, 1])
        training_data['X'].append(sigs[sub][sess][inds])

    # Flatten across sessions
    X_array = np.concatenate(training_data['X'])
    y_array = np.concatenate(training_data['labels'])

    # Normalize
    for i in range(n_channels):
        X_array[:, i] = normalize(X_array[:, i])

    # Ensure even samples per fold
    n_per_fold = int(np.floor(216/kfolds))
    X_array = X_array[:int(n_per_fold * kfolds)]
    y_array = y_array[:int(n_per_fold * kfolds)]

    # Randomly shuffle
    np.random.seed(0)
    inds = np.arange(len(X_array))
    np.random.shuffle(inds)

    X_array = X_array[inds]
    y_array = y_array[inds]

    # K-folds
    #X_array = X_array[:, :, int(fs):int(3.5 * fs)]
    X_array = X_array.reshape(kfolds, -1, 128, n_samples)
    y_array = y_array.reshape(kfolds, -1)

    train_inds = np.array(list(combinations(np.arange(kfolds), 4)))
    test_inds = np.arange(5)[::-1]

    # To tensors
    X_tensor = torch.from_numpy(X_array.astype(np.float32))
    y_tensor = torch.from_numpy(y_array)

    # Test
    for k, (itrain, itest) in enumerate(zip(train_inds, test_inds)):

        # Reshape into train and test
        X_test = X_tensor[itest]
        X_test = X_test.reshape(-1, n_channels, n_samples)

        y_test = y_tensor[itest]
        y_test = y_test.reshape(-1)
        
        model = torch.load(f'models_deep_copy4/convnet_{sub}_best_kfold{str(k)}')
        y_pred = torch.argmax(model(X_test), axis=1)
        
        accuracy[isub, k] = len(torch.where(y_pred == y_test)[0]) / len(X_test)
        
        precision[isub, k], recall[isub, k], fscore[isub, k], _ = \
            precision_recall_fscore_support(y_test, y_pred, average='macro')
        
        # Per class metrics
        for iclass in range(4):
            inds = np.where(y_test == iclass)[0]

            _y_true = y_test[inds]
            _y_pred = y_pred[inds]
            
            acc = len(torch.where(_y_true == _y_pred)[0]) / len(_y_true)
            accuracy_class[isub, k, iclass] = acc
            
        precision_class[isub, k], recall_class[isub, k], fscore_class[isub, k], _ = \
            precision_recall_fscore_support(y_test, y_pred)

In [107]:
df = pd.DataFrame()

df['Accuracy'] = [f'{i:.2f}%' for i in accuracy.mean(axis=1).round(4) * 100]
df['Recall'] = [f'{i:.2f}%' for i in recall.mean(axis=1).round(4) * 100]
df['Precision'] = [f'{i:.2f}%' for i in precision.mean(axis=1).round(4) * 100]
df['F-Score'] = [f'{i:.2f}%' for i in fscore.mean(axis=1).round(4) * 100]

df.append([
    f'{accuracy.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{recall.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{precision.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{fscore.mean(axis=1).mean().round(4) * 100:.2f}%'
], ignore_index=True)


df.loc[len(df)] = [
    f'{accuracy.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{recall.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{precision.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{fscore.mean(axis=1).mean().round(4) * 100:.2f}%'
]


labels = [f'sub-{str(i).zfill(2)}' for i in range(10)]
labels.append('Average')

df.insert(0, 'Subject', labels)

df

/var/folders/xq/64b6njb134n_vwp47x74rp240000gn/T/ipykernel_29314/3447570151.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([


,Subject,Accuracy,Recall,Precision,F-Score
0,sub-00,27.00%,26.50%,23.66%,23.48%
1,sub-01,43.72%,44.47%,45.88%,41.56%
2,sub-02,28.33%,29.22%,31.27%,27.51%
3,sub-03,33.95%,33.66%,35.67%,32.54%
4,sub-04,44.65%,43.60%,44.03%,42.76%
5,sub-05,25.12%,26.34%,22.71%,22.10%
6,sub-06,44.19%,45.02%,47.78%,43.17%
7,sub-07,27.00%,29.97%,27.06%,25.21%
8,sub-08,41.86%,41.10%,44.47%,38.81%
9,sub-09,45.58%,45.90%,45.80%,43.21%


## Correction

Above and in Berg, Donkelaar, Alimardani 2021, the entire trial was used for training and testing. This includes fixation periods and visual stimuli presentation. Below, this is correct, using only the action period of the task.

In [7]:
n_channels = 128
n_samples = 640 #1100

kfolds = 5

# Macro
accuracy = np.zeros((10, 5))
precision = np.zeros((10, 5))
recall = np.zeros((10, 5))
fscore = np.zeros((10, 5))

# Per class metrics
accuracy_class = np.zeros((10, 5, 4))
precision_class = np.zeros((10, 5, 4))
recall_class = np.zeros((10, 5, 4))
fscore_class = np.zeros((10, 5, 4))

# Train models separately for each subject
for isub, sub in enumerate(subjects.keys()):

    # Load data
    training_data = {'X': [], 'labels': []}

    for sess in sessions:

        events = np.load(subjects[sub][sess]['events'], allow_pickle=True)

        inds = np.where(events[:, 2] == 1)[0]

        training_data['labels'].append(events[inds][:, 1])
        training_data['X'].append(sigs[sub][sess][inds])

    # Flatten across sessions
    X_array = np.concatenate(training_data['X'])
    y_array = np.concatenate(training_data['labels'])

    # Normalize
    for i in range(n_channels):
        X_array[:, i] = normalize(X_array[:, i])

    # Ensure even samples per fold
    n_per_fold = int(np.floor(216/kfolds))
    X_array = X_array[:int(n_per_fold * kfolds)]
    y_array = y_array[:int(n_per_fold * kfolds)]

    # Randomly shuffle
    np.random.seed(0)
    inds = np.arange(len(X_array))
    np.random.shuffle(inds)

    X_array = X_array[inds]
    y_array = y_array[inds]

    # K-folds
    X_array = X_array[:, :, int(fs):int(3.5 * fs)]
    X_array = X_array.reshape(kfolds, -1, 128, n_samples)
    y_array = y_array.reshape(kfolds, -1)

    train_inds = np.array(list(combinations(np.arange(kfolds), 4)))
    test_inds = np.arange(5)[::-1]

    # To tensors
    X_tensor = torch.from_numpy(X_array.astype(np.float32))
    y_tensor = torch.from_numpy(y_array)

    # Test
    for k, (itrain, itest) in enumerate(zip(train_inds, test_inds)):

        # Reshape into train and test
        X_test = X_tensor[itest]
        X_test = X_test.reshape(-1, n_channels, n_samples)

        y_test = y_tensor[itest]
        y_test = y_test.reshape(-1)
        
        model = torch.load(f'models_deep_copy4_corrected_gpu/models/convnet_{sub}_best_kfold{str(k)}')
        y_pred = torch.argmax(model(X_test), axis=1)
        
        accuracy[isub, k] = len(torch.where(y_pred == y_test)[0]) / len(X_test)
        
        precision[isub, k], recall[isub, k], fscore[isub, k], _ = \
            precision_recall_fscore_support(y_test, y_pred, average='macro')
        
        # Per class metrics
        for iclass in range(4):
            inds = np.where(y_test == iclass)[0]

            _y_true = y_test[inds]
            _y_pred = y_pred[inds]
            
            acc = len(torch.where(_y_true == _y_pred)[0]) / len(_y_true)
            accuracy_class[isub, k, iclass] = acc
            
        precision_class[isub, k], recall_class[isub, k], fscore_class[isub, k], _ = \
            precision_recall_fscore_support(y_test, y_pred)

/Users/ryanhammonds/projects/bci/.env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ryanhammonds/projects/bci/.env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
df = pd.DataFrame()

df['Accuracy'] = [f'{i:.2f}%' for i in accuracy.mean(axis=1).round(4) * 100]
df['Recall'] = [f'{i:.2f}%' for i in recall.mean(axis=1).round(4) * 100]
df['Precision'] = [f'{i:.2f}%' for i in precision.mean(axis=1).round(4) * 100]
df['F-Score'] = [f'{i:.2f}%' for i in fscore.mean(axis=1).round(4) * 100]

df.append([
    f'{accuracy.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{recall.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{precision.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{fscore.mean(axis=1).mean().round(4) * 100:.2f}%'
], ignore_index=True)


df.loc[len(df)] = [
    f'{accuracy.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{recall.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{precision.mean(axis=1).mean().round(4) * 100:.2f}%',
    f'{fscore.mean(axis=1).mean().round(4) * 100:.2f}%'
]


labels = [f'sub-{str(i).zfill(2)}' for i in range(10)]
labels.append('Average')

df.insert(0, 'Subject', labels)

/var/folders/xq/64b6njb134n_vwp47x74rp240000gn/T/ipykernel_31167/3447570151.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([


,Subject,Accuracy,Recall,Precision,F-Score
0,sub-00,26.00%,25.41%,24.16%,23.04%
1,sub-01,35.81%,36.32%,43.55%,34.44%
2,sub-02,23.33%,25.07%,24.38%,22.36%
3,sub-03,35.35%,36.65%,36.51%,34.17%
4,sub-04,33.95%,34.29%,32.23%,31.41%
5,sub-05,20.00%,20.16%,21.61%,19.91%
6,sub-06,37.21%,38.43%,39.00%,35.58%
7,sub-07,22.50%,23.84%,21.38%,19.94%
8,sub-08,41.40%,41.72%,42.68%,39.65%
9,sub-09,37.67%,38.00%,39.09%,36.08%
